# MCP Discovery Tool for LangChain

This notebook demonstrates how to use the MCP Discovery tool to enable LangChain agents to dynamically discover and select tools based on their task requirements.

## What is MCP Discovery?

MCP Discovery provides a semantic search API over Model Context Protocol (MCP) servers, allowing agents to:
- Find relevant tools based on natural language queries
- Compare server performance metrics
- Get installation instructions automatically
- Discover new capabilities dynamically

## Installation

```bash
pip install langchain langchain-openai requests
```

In [ ]:
# Import required libraries
from mcp_discovery_tool import MCPDiscoveryTool, create_mcp_discovery_tool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
import os

## Example 1: Basic Usage

Simply use the tool to discover MCP servers based on a capability description.

In [ ]:
# Create the discovery tool
discovery_tool = create_mcp_discovery_tool()

# Search for database-related MCP servers
result = discovery_tool.run("PostgreSQL database access")
print(result)

## Example 2: Different Search Queries

The tool supports various types of capability searches.

In [ ]:
# Search for different capabilities
queries = [
    "send emails",
    "file system operations",
    "weather data API",
    "web scraping",
    "cloud storage"
]

for query in queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print('='*60)
    result = discovery_tool.run(query)
    print(result)

## Example 3: Agent with MCP Discovery

Integrate MCP Discovery into a LangChain agent so it can find tools dynamically.

In [ ]:
# Set up OpenAI API key (required for agent)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Create LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Create tools list with MCP Discovery
tools = [discovery_tool]

# Get the ReAct prompt from hub
prompt = hub.pull("hwchase17/react")

# Create agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# Example: Agent discovers database tools when needed
response = agent_executor.invoke({
    "input": "I need to work with a PostgreSQL database. What tools are available?"
})

print("\nAgent Response:")
print(response["output"])

## Example 4: Using Performance Metrics for Selection

The tool returns performance metrics that agents can use for decision-making.

In [ ]:
# Agent compares multiple options based on metrics
response = agent_executor.invoke({
    "input": (
        "I need to send emails. Find me the best email MCP server "
        "with the lowest latency and highest uptime."
    )
})

print("\nAgent Response:")
print(response["output"])

## Example 5: Custom API Endpoint

You can point to a custom MCP Discovery API instance if needed.

In [ ]:
# Use custom API endpoint (e.g., for testing or private deployments)
custom_tool = create_mcp_discovery_tool(
    api_url="https://your-custom-mcp-discovery.example.com"
)

result = custom_tool.run("file storage")
print(result)

## Example 6: Error Handling

The tool gracefully handles API errors and timeouts.

In [ ]:
# Example: No matching servers found
result = discovery_tool.run("extremely obscure nonexistent capability xyz123")
print(result)

# The agent can handle this and adjust its approach
response = agent_executor.invoke({
    "input": "Find me a tool for quantum encryption. If none exists, suggest alternatives."
})
print("\nAgent Response:")
print(response["output"])

## Example 7: Multi-Step Agent Workflow

An agent can discover multiple tools in sequence based on a complex task.

In [ ]:
# Complex task requiring multiple tool discoveries
response = agent_executor.invoke({
    "input": (
        "I need to build a workflow that: "
        "1) Fetches data from a database, "
        "2) Processes it with file operations, and "
        "3) Sends the results via email. "
        "What MCP servers should I use for each step?"
    )
})

print("\nAgent Response:")
print(response["output"])

## Key Takeaways

1. **Dynamic Tool Discovery**: Agents can find tools based on task needs rather than pre-configured lists
2. **Performance Metrics**: Make informed decisions based on latency and uptime data
3. **Installation Info**: Get immediate setup instructions for discovered servers
4. **Semantic Search**: Natural language queries work well for capability matching
5. **Error Resilient**: Graceful handling of missing results or API issues

## Next Steps

- Explore the [MCP Discovery API documentation](https://github.com/yksanjo/mcp-discovery)
- Browse available MCP servers at the [MCP Registry](https://github.com/mcp)
- Integrate with your existing LangChain agents
- Contribute additional MCP servers to expand the ecosystem